https://www.meetup.com/PyData-PDX/events/280454606

http://xarray.pydata.org/en/stable/

# Let's learn xarray by answering some questions with weather data:

 - Q1. How hot did it get in Portland, Oregeon this summer?
 - Q2. How much hotter was that compared to last year?
 - Q3. During the peak of the heatwave where else in the world was hotter than Portland?

# Get a temperature dataset for Portland (xarray advanced)

In [ ]:
files = []

import xarray as xr

for dt in xr.cftime_range("2020-01", "2021-08", freq="M"):
    files.append(
        f"s3://era5-pds/zarr/{dt:%Y}/{dt:%m}/data/air_temperature_at_2_metres.zarr"
    )

from geopy.geocoders import Photon

geolocator = Photon()
location = geolocator.geocode("Portland, Oregon")
print(location)
lat = location.point.latitude
lon = location.point.longitude + 360
print(lat, lon)


def select_location(ds: xr.Dataset) -> xr.Dataset:
    return ds.sel(lat=lat, lon=lon, method="nearest")


import cf_xarray.units
import pint_xarray
from tqdm.dask import TqdmCallback

TqdmCallback(desc="dask tasks").register()

portland_ds = (
    xr.open_mfdataset(
        files,
        engine="zarr",
        backend_kwargs=dict(storage_options={"anon": True}),
        parallel=True,
        preprocess=select_location,
    )
    .compute()
    .pint.quantify()
    .pint.to("degF")
)
portland_ds = portland_ds.pint.dequantify(format="~P")

from timezonefinder import TimezoneFinder

tz = TimezoneFinder().timezone_at(lng=location.point.longitude, lat=lat)
print(tz)
import cf_xarray

time_dim_name = portland_ds.cf["time"].name
time_dim_attrs = portland_ds.cf["time"].attrs
portland_ds = portland_ds.assign_coords(
    {
        time_dim_name: portland_ds[time_dim_name]
        .to_index()
        .tz_localize("UTC")
        .tz_convert(tz)
        .tz_localize(None)
        .to_numpy()
    }
)
portland_ds[time_dim_name].attrs = time_dim_attrs

In [ ]:
type(portland_ds)

http://xarray.pydata.org/en/stable/generated/xarray.Dataset.html

In [ ]:
portland_ds

Firstly, you can transform the data to a format you are more familar with:

In [ ]:
df = portland_ds.reset_coords(drop=True).to_dataframe()
print(type(df))
df

In [ ]:
arr = portland_ds["air_temperature_at_2_metres"].values
print(type(arr))
arr

You can also do the inverse of the transformations:

In [ ]:
df.to_xarray()

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_xarray.html

In [ ]:
da = xr.DataArray(arr)
da

http://xarray.pydata.org/en/stable/generated/xarray.DataArray.html

In [ ]:
da.to_dataset(name="arr")

# Q1. How hot did it get in Portland, Oregeon this summer?

Visualize the data using xarray's plotting capability (http://xarray.pydata.org/en/stable/user-guide/plotting.html):

In [ ]:
portland_ds["air_temperature_at_2_metres"].plot()

Create a quick interactive plot using hvplot (https://hvplot.holoviz.org/)

In [ ]:
import hvplot.xarray

portland_ds.hvplot()

In [ ]:
portland_ds.max()

Query a Dataset on a value

In [ ]:
hot_ds = portland_ds.where(portland_ds == portland_ds.max(), drop=True)
hot_ds

## A1. How hot did it get in Portland, Oregeon this summer?

In [ ]:
ts = hot_ds["time0"].to_index().tz_localize(tz).to_pydatetime()[0]
max_temp = hot_ds["air_temperature_at_2_metres"].values[0]
units = hot_ds["air_temperature_at_2_metres"].attrs["units"]
print(
    f"The max tempetaure in Portland, Oregeon this summer was {max_temp:.2f} {units} on {ts}"
)

# Q2. How much hotter was that compared to last year?

In [ ]:
da = portland_ds["air_temperature_at_2_metres"]
da

In [ ]:
g = da.groupby("time0.year").max()
g

http://xarray.pydata.org/en/stable/user-guide/groupby.html

In [ ]:
g.sel(year=2021) - g.sel(year=2020)

http://xarray.pydata.org/en/stable/user-guide/indexing.html?highlight=indexing

## A2. How much hotter was that compared to last year?

In [ ]:
diff = (g.sel(year=2021) - g.sel(year=2020)).values

from sklearn.metrics import mean_absolute_percentage_error

mape = (
    mean_absolute_percentage_error(
        [g.sel(year=2021).to_numpy()],
        [g.sel(year=2020).to_numpy()],
    )
    * 100
)

print(f"The peak temeperature different between 2021 and 2020 was {diff:.2f} {units}")
print(f"That's {mape:.2f} percent hotter")

# Q3. During the peak of the heatwave where else in the world was hotter than Portland?

In [ ]:
import pytz

dt_utc = ts.astimezone(pytz.timezone("UTC")).replace(tzinfo=None)

da_ts = (
    xr.open_zarr(
        f"s3://era5-pds/zarr/{dt_utc:%Y}/{dt_utc:%m}/data/air_temperature_at_2_metres.zarr",
        storage_options={"anon": True},
    )["air_temperature_at_2_metres"].sel(time0=dt_utc)
).compute()

Convert degrees Kelvin to degrees Fahrenheit using https://pint-xarray.readthedocs.io/en/latest/

In [ ]:
da_ts = da_ts.pint.quantify().pint.to("degF").pint.dequantify(format="~P")

Convert longitude from 0 - 360 to -180 - 180

In [ ]:
with xr.set_options(keep_attrs=True):
    da_ts = da_ts.assign_coords({"lon": (((da_ts["lon"] + 180) % 360) - 180)}).sortby(
        "lon"
    )

Reindex latitude to from 90 - -90 to -90 - 90 

In [ ]:
da_ts = da_ts.reindex({"lat": da_ts["lat"][::-1]})
da_ts

Check the data was read in correctly:

In [ ]:
da_ts.sel(lat=lat, lon=lon - 360, method="nearest")

Plot the spatial data using cartopy:

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

plt.figure(figsize=(16, 8))
ax = plt.subplot(projection=ccrs.PlateCarree())
da_ts.plot(ax=ax, cmap=plt.cm.viridis)
ax.coastlines()
ax.gridlines(draw_labels=True);

Create an interactive map using Geoviews https://geoviews.org/index.html

In [ ]:
import geoviews as gv

gv_da_ts = gv.Dataset(da_ts)
gv_img = gv_da_ts.to(gv.Image, kdims=["lon", "lat"])

from holoviews.operation.datashader import rasterize

gv_raster = rasterize(gv_img)

# Download a populous city dataset
import geopandas as gpd

cities = gpd.read_file(
    "https://naturalearth.s3.amazonaws.com/10m_cultural/ne_10m_populated_places.zip"
)[["NAME", "geometry"]]
gv_pts = gv.Points(cities)

# Overlay layers: raster (temperature), coastlines, country boarders, state boarders, capital cities
# with custom plotting options
import geoviews.feature as gf

gv_raster.opts(
    tools=["hover"],
    width=1000,
    height=600,
    cmap=plt.cm.viridis.name,
    colorbar=True,
    colorbar_position="right",
) * gf.coastline() * gf.borders() * gf.states(color=None) * gv_pts.opts(
    tools=["hover"], color="black", size=2
)

Flatten the 2d array to 1d

In [ ]:
da_ts_1d = da_ts.stack({"point": ["lat", "lon"]})
da_ts_1d

sort values

In [ ]:
da_ts_1d_sorted = da_ts_1d.sortby(da_ts_1d)
da_ts_1d_sorted

Keep values greater than the temperature in portland

In [ ]:
da_ts_1d_sorted.where(da_ts_1d_sorted >= max_temp, drop=True)

In [ ]:
points_hotter_than_portland = da_ts_1d_sorted.where(
    da_ts_1d_sorted >= max_temp, drop=True
)
points_hotter_than_portland

## A3. During the peak of the heatwave where else in the world was hotter than Portland?

In [ ]:
locs = []
for p in points_hotter_than_portland["point"].values:
    loc = geolocator.reverse(p)
    if "Portland" not in str(loc):
        print("y")
        locs.append(loc)
if len(locs) == 0:
    print("Nowhere!")